## Imports

In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Convolution2D, Activation, MaxPooling2D, Dense
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

## Datagen object creation

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
##image aug datagen
datagen_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255., # Rescaling
    rotation_range = 40, # for augmentation
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_dir = 'C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/train'
test_dir = 'C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/test'

In [3]:
image_size = [256, 256]

In [4]:
images, labels = next(datagen_aug.flow_from_directory(train_dir))

Found 1239 images belonging to 34 classes.


In [5]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 34)


## Getting test and train len

In [ ]:
train_image_ds = tf.data.Dataset.list_files(train_dir+ '/*/*', shuffle=False)
test_image_ds = tf.data.Dataset.list_files(test_dir + '/*/*', shuffle=False)

In [ ]:
test_img_count=len(test_image_ds)
train_img_count = len(train_image_ds)

## Loading image with optimized data pipeline

In [ ]:
training_set = tf.data.Dataset.from_generator(
    lambda: datagen_aug.flow_from_directory(train_dir), 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([None,256,256,3], [None,34])
)

training_set.element_spec

In [ ]:
test_set = tf.data.Dataset.from_generator(
    lambda: test_datagen.flow_from_directory(test_dir), 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([None,256,256,3], [None,34])
)

test_set.element_spec

## Build Model

In [ ]:
resnet = tf.keras.applications.ResNet50V2(
    input_shape=image_size + [3],
    include_top=False,
    weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
)

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(labels)+2, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
start = time.time()
history = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=train_img_count,
  validation_steps=test_img_count
)
end = time.time()
print("Time taken:", end - start)

In [ ]:

# plot the loss
# plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
# plt.legend()
# plt.show()
# plt.savefig('LossVal_loss')

# # plot the accuracy
# plt.plot(r.history['accuracy'], label='train acc')
# plt.plot(r.history['val_accuracy'], label='val acc')
# plt.legend()
# plt.show()
# plt.savefig('AccVal_acc')